In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

chromedriver.exe 경로 수정

In [2]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe' 

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

네이버에 언론사 설정하고 검색 시에 url_domain + 검색어 + news_domain 과 같은 구조  
그래서 아래 함수에서 news_domain을 각자 언론사에 맞춰서 수정해주시면 됩니다

경향신문

In [30]:
def biz_khan(query,driver_path):
    domain = '경향신문'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1032&nso=&is_sug_officeid=0' 
    url = url_domain + query + news_domain
    
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 기사 url을 list형태로 반환
    try:
        news_list = soup.select('#main_pack > section > div > div.group_news > ul')
        url_list = []
        news_list = news_list[0].find_all('li')
        for url in news_list:
            url_list.append(str(url.find_all('a')[0].get('data-url')))
    except:
        print('기사가 없습니다.')
        driver.close()
        return 0
    driver.close()
    
    head_list = []
    data_list = []
    
    for url in url_list:
        driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
        driver.get(url)
        time.sleep(2)
        # try문 안에서 각자 언론사에 맞게 수정
        try:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head_list.append(soup.find('h1',id='articleTtitle').text)
            tags = soup.select_one('#container > div.main_container > div.art_cont > div.art_body').find_all('p')
            data = [clean_text(x.text) for x in tags]
            data_list.append(' '.join(data))
        except:
            print(url)
            print('None_type')
        driver.close()
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('삼성전자 슬로건',driver_path)

http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201703210802011&code=620109
None_type


중앙일보

In [5]:
def chungang(query,driver_path):
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows

    domain = '중앙일보'
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1025&nso=&is_sug_officeid=0'
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')

    # 기사 url을 list형태로 반환
    try:
        news_list = soup.select('#main_pack > section > div > div.group_news > ul')
        url_list = []
        news_list = news_list[0].find_all('li')
        for url in news_list:
            url_list.append(str(url.find_all('a')[0].get('data-url')))
    except:
        print('기사가 없습니다.')
        driver.close()
        return 0
    driver.close()

    head_list = []
    data_list = []

    for url in url_list:
        try:
            driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
            driver.get(url)
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head = clean_text(soup.select('#article_title')[0].text)
            head_list.append(head)
            text = soup.find('div', id='article_body').text
            text = clean_text(text)
            data_list.append(text)
        except:
            print("None_type")
        driver.close()
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

chungang_df = chungang('삼성전자 슬로건', driver_path)

In [6]:
chungang_df

,언론사,제목,내용
0,중앙일보,삼성· 펄펄 나는데 ‘탱크주의’ 대우전자가 아쉬운 이유,1993년 대우전자가 선보인 탱크주의 신문 광고 배순훈 대우전 사장오른쪽 광고 속 ...
1,중앙일보,코로나 위기로 브랜드 더 중요해져…딤채·삼성서울병원 등 1위,사진 브랜드스탁 신종 코로나바이러스 감염증코로나19의 확산 속에도 딤채 하나투어 제...
2,중앙일보,“비스포크 냉장고 130만원 올레드 200만원”…삼성·전자 31일까지 세일 행사,삼성전자와 전자가 이달 31일까지 한 달간 주요 가전제품과 모바일기기를 대상으로 세...
3,중앙일보,2021 승현준 삼성리서치 사장 “맞춤형 가전으로 ‘보다 나은 일상’ 실현”,승현준 삼성전자 사장삼성리서치 소장이 11일현지시간 온라인으고 개막한 2021 삼성...
4,중앙일보,미 헤지펀드 삼성에 밀린 반도체 생산 멈춰라…인텔의 위기,삼성전자와 인텔 로고 중앙포토 1990년대 중반 이후 ‘인텔 인사이드’라는 슬로건과...
5,중앙일보,바이든 승리에도 삼성 美 텍사스에 반도체 추가투자 가능성,삼성 미국 오스틴 팹 사진 삼성전자 7일현지시간 미국 대선에서 승리 선언을 한 조지...
6,중앙일보,스마트폰 시장 긁지않은 복권…삼성·애플 14억 인도 쟁탈전,지난8월 인도 벵갈루루에 위치한 삼성 오페라 하우스에서 갤럭시 노트20를 체험하고 ...
7,중앙일보,파운드리 2위 삼성에 는 넘사벽,삼성전자 평택캠퍼스 전경 사진 삼성전자 가까이하기엔 너무 멀다 파운드리반도체 위탁생...
8,중앙일보,파운드리 2위 삼성 공격 투자에도 와 격차 커진 까닭,삼성 가까이하기엔 너무 멀다 파운드리반도체 위탁생산 시장에서 1위를 지키고 있는 대...
9,중앙일보,윤석열 “파일은 마타도어…검증은 팩트에 기초해 이뤄져야,윤석열 전 검찰총장은 29일 출마선언 직후 곧바로 기자들의 질문 검증대에 섰다 약 ...


조선일보

In [7]:
def joseon(query,driver_path):
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows

    domain = '조선일보'
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1023&nso=&is_sug_officeid=0'
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')

    # 기사 url을 list형태로 반환
    try:
        news_list = soup.select('#main_pack > section > div > div.group_news > ul')
        url_list = []
        news_list = news_list[0].find_all('li')
        for url in news_list:
            url_list.append(str(url.find_all('a')[0].get('data-url')))
    except:
        print('기사가 없습니다.')
        driver.close()
        return 0
    driver.close()
    
    head_list = []
    data_list = []

    for url in url_list:
        try:
            driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
            driver.get(url)
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head = clean_text(soup.find_all("h1", class_="article-header__headline | font--primary text--black")[0].text)
            head_list.append(head)
            tags = soup.find_all('p',class_=' article-body__content article-body__content-text | text--black text font--size-sm-18 font--size-md-18 font--primary')
            data = [clean_text(x.text) for x in tags]
            data_list.append(' '.join(data))
        except:
            print("None_type")
        driver.close()
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

joseon_df = joseon('삼성전자 슬로건', driver_path)

In [8]:
joseon_df

,언론사,제목,내용
0,조선일보,이제는 가전을 나답게 삼성전자 맞춤형 가전 강화,‘이제는 가전을 나답게’ 삼성전자가 새롭게 정한 가전제품 통합 슬로건이다 삼성전자는...
1,조선일보,지펠·디오스·쏘나타… 별나고 멋진 중산층 만든 디자인,이 글의 제목은 영국의 팝아트 작가 리처드 해밀턴의 1956년 작 ‘오늘의 가정을 ...
2,조선일보,“정치와 정책의 괴리…文 정부는 정경환란 상태”,한국 경제의 역사를 쉽게 이해하는 방법 중의 하나는 역대 대통령들의 경제정책을 분석...
3,조선일보,냉장고 내부도 이젠 내 마음대로 삼성전자 뉴 셰프컬렉션 출시,삼성전자가 3번째 프로젝트 프리즘맞춤형 가전 제품인 ‘뉴 셰프컬렉션 냉장고’를 2일...
4,조선일보,자산 364억 빚 291억공약만큼 재산도 파격적인 ‘이단아’ 허경영,“나라에 돈이 없는 것이 아니라 도둑놈이 많습니다” 지난달 28일 오전 10시 서울...
5,조선일보,미중 사이 줄타기하며 성장한 미중 갈등 핵심으로,미·중 갈등이 다시 달아오르면서 최근 가장 뜨겁게 주목받는 반도체 기업이 있다 대만...
6,조선일보,세계 파운드리 54 장악한 대만 킹핀… 美·中 갈등의 핵으로,미·중 갈등이 다시 달아오르면서 최근 가장 뜨겁게 주목받는 반도체 기업이 있다 대만...
7,조선일보,2020 의 모든 것 보여준다,인공지능를 우리의 일상으로 내년 1월 710일현지 시각 미국 라스베이거스에서 미국소...
8,조선일보,가장 친환경적경제적 에너지원 에너지 효율 혁신 전략 한눈에,대한민국의 에너지 산업과 기술의 현재와 미래를 한눈에 살펴볼 수 있는 전시회가 열린...
9,조선일보,스토리 대만을 대표하는 기업가 궈타이밍은 ‘트럼프’가 될까,궈타이밍郭台銘·69·사진은 대만의 트럼프가 될 수 있을 것인가 세계 최대 전자제품 ...


전국매일신문

In [9]:
def alldays(query,driver_path):
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows

    domain = '전국매일신문'
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2844&nso=&is_sug_officeid=0'
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')

    # 기사 url을 list형태로 반환
    try:
        news_list = soup.select('#main_pack > section > div > div.group_news > ul')
        url_list = []
        news_list = news_list[0].find_all('li')
        for url in news_list:
            url_list.append(str(url.find_all('a')[0].get('data-url')))
    except:
        print('기사가 없습니다.')
        driver.close()
        return 0
    
    driver.close()

    head_list = []
    data_list = []

    for url in url_list:
        try:
            driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
            driver.get(url)
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head = clean_text(soup.find('div',class_="article-head-title").text)
            head_list.append(head)
            tags = soup.select_one('#article-view-content-div').find_all('p')
            data = [clean_text(x.text) for x in tags]
            data_list.append(' '.join(data))
        except:
            print("None_type")
        driver.close()
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

alldays_df = alldays('삼성전자 슬로건', driver_path)

기사가 없습니다.


In [26]:
biz

,언론사,제목,내용
0,경향신문,"삼성전자, 21일 밤 국내외 사업장서 ‘전등끄기’ 까닭은",삼성전자가 지난 21일 저녁 경기 수원 삼성 디지털시티와 44개 해외 판매·생산법인...
1,경향신문,삼성 가전 새 슬로건 ‘이제는 가전을 나답게’,ㆍ‘틀에 박힌 시장에 마침표’ 의미 담아…마케팅 전반에 활용 예정 삼성전자가 전 가...
2,경향신문,"철 없는 ‘4K TV’, 삼성에 효자 TV",게임 환경에서 4 왼쪽와 일반 의 화질을 비교한 모습 삼성전자 제공 신제품 ‘8 ’...
3,경향신문,"[단독] 윤종원 청와대 경제수석 첫 인터뷰 “문 대통령·삼성 이재용 만남, 재벌 구...",·윤종원 신임 경제수석 임명 후 첫 언론 인터뷰 ·대통령이 삼성 이재용 부회장 만난...
4,경향신문,삼성그룹 새해 슬로건 ‘새로운 도전’···경영기조는 ‘내실’,삼성그룹이 새해 경영전략의 기조를 ‘내실 다지기’로 정했다 ‘새로운 도전’을 슬로건...
5,경향신문,삼성 “당신 일상의 새 기준”…LG “더 나은 삶으로 혁신”,ㆍ국제가전전시회 내일 개막 삼성전자가 다음달 1일 독일 베를린에서 개막하는 ‘국제가...
6,경향신문,무풍 에어컨의 돌풍?···삼성 국내 에어컨 판매 100만대 돌파,삼성전자가 ‘무풍 에어컨’ 인기에 힘입어 올해 들어 가정용 에어컨 국내 판매 100...
7,경향신문,삼성전자 ‘스마트홈’ 사업 美 CES서 선보여,삼성전자는 ·냉장고·세탁기 등 집안의 가전기기에다 스마트폰 같은 기기를 연결해 손쉽...
8,경향신문,[기업특집] “실패해도 괜찮아”…삼성의 도전은 계속된다,삼성전자 수원공장에는 스페이스와 아이디어 라운지라는 다소 이색적인 공간이 마련돼 있...


In [27]:
chungang_df

,언론사,제목,내용
0,중앙일보,삼성· 펄펄 나는데 ‘탱크주의’ 대우전자가 아쉬운 이유,1993년 대우전자가 선보인 탱크주의 신문 광고 배순훈 대우전 사장오른쪽 광고 속 ...
1,중앙일보,코로나 위기로 브랜드 더 중요해져…딤채·삼성서울병원 등 1위,사진 브랜드스탁 신종 코로나바이러스 감염증코로나19의 확산 속에도 딤채 하나투어 제...
2,중앙일보,“비스포크 냉장고 130만원 올레드 200만원”…삼성·전자 31일까지 세일 행사,삼성전자와 전자가 이달 31일까지 한 달간 주요 가전제품과 모바일기기를 대상으로 세...
3,중앙일보,2021 승현준 삼성리서치 사장 “맞춤형 가전으로 ‘보다 나은 일상’ 실현”,승현준 삼성전자 사장삼성리서치 소장이 11일현지시간 온라인으고 개막한 2021 삼성...
4,중앙일보,미 헤지펀드 삼성에 밀린 반도체 생산 멈춰라…인텔의 위기,삼성전자와 인텔 로고 중앙포토 1990년대 중반 이후 ‘인텔 인사이드’라는 슬로건과...
5,중앙일보,바이든 승리에도 삼성 美 텍사스에 반도체 추가투자 가능성,삼성 미국 오스틴 팹 사진 삼성전자 7일현지시간 미국 대선에서 승리 선언을 한 조지...
6,중앙일보,스마트폰 시장 긁지않은 복권…삼성·애플 14억 인도 쟁탈전,지난8월 인도 벵갈루루에 위치한 삼성 오페라 하우스에서 갤럭시 노트20를 체험하고 ...
7,중앙일보,파운드리 2위 삼성에 는 넘사벽,삼성전자 평택캠퍼스 전경 사진 삼성전자 가까이하기엔 너무 멀다 파운드리반도체 위탁생...
8,중앙일보,파운드리 2위 삼성 공격 투자에도 와 격차 커진 까닭,삼성 가까이하기엔 너무 멀다 파운드리반도체 위탁생산 시장에서 1위를 지키고 있는 대...
9,중앙일보,윤석열 “파일은 마타도어…검증은 팩트에 기초해 이뤄져야,윤석열 전 검찰총장은 29일 출마선언 직후 곧바로 기자들의 질문 검증대에 섰다 약 ...


In [28]:
joseon_df

,언론사,제목,내용
0,조선일보,이제는 가전을 나답게 삼성전자 맞춤형 가전 강화,‘이제는 가전을 나답게’ 삼성전자가 새롭게 정한 가전제품 통합 슬로건이다 삼성전자는...
1,조선일보,지펠·디오스·쏘나타… 별나고 멋진 중산층 만든 디자인,이 글의 제목은 영국의 팝아트 작가 리처드 해밀턴의 1956년 작 ‘오늘의 가정을 ...
2,조선일보,“정치와 정책의 괴리…文 정부는 정경환란 상태”,한국 경제의 역사를 쉽게 이해하는 방법 중의 하나는 역대 대통령들의 경제정책을 분석...
3,조선일보,냉장고 내부도 이젠 내 마음대로 삼성전자 뉴 셰프컬렉션 출시,삼성전자가 3번째 프로젝트 프리즘맞춤형 가전 제품인 ‘뉴 셰프컬렉션 냉장고’를 2일...
4,조선일보,자산 364억 빚 291억공약만큼 재산도 파격적인 ‘이단아’ 허경영,“나라에 돈이 없는 것이 아니라 도둑놈이 많습니다” 지난달 28일 오전 10시 서울...
5,조선일보,미중 사이 줄타기하며 성장한 미중 갈등 핵심으로,미·중 갈등이 다시 달아오르면서 최근 가장 뜨겁게 주목받는 반도체 기업이 있다 대만...
6,조선일보,세계 파운드리 54 장악한 대만 킹핀… 美·中 갈등의 핵으로,미·중 갈등이 다시 달아오르면서 최근 가장 뜨겁게 주목받는 반도체 기업이 있다 대만...
7,조선일보,2020 의 모든 것 보여준다,인공지능를 우리의 일상으로 내년 1월 710일현지 시각 미국 라스베이거스에서 미국소...
8,조선일보,가장 친환경적경제적 에너지원 에너지 효율 혁신 전략 한눈에,대한민국의 에너지 산업과 기술의 현재와 미래를 한눈에 살펴볼 수 있는 전시회가 열린...
9,조선일보,스토리 대만을 대표하는 기업가 궈타이밍은 ‘트럼프’가 될까,궈타이밍郭台銘·69·사진은 대만의 트럼프가 될 수 있을 것인가 세계 최대 전자제품 ...


In [29]:
alldays_df

0

* * *  
저장할 때

In [ ]:
# biz.to_csv('경향신문.csv',encoding='CP949',index=None)

In [ ]:
# test = pd.read_csv('경향신문.csv',encoding='CP949')

In [ ]:
# test

* * * 
연습